# Modified Gale Shapley Algorithm

In [10]:
import pandas as pd

In [7]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [27]:
syndata.iloc[0] #Use this to access a row
syndata.loc[0, :] #Use this to access a row
x = syndata.loc[:,"Age"] #Use this to access a column

Now we work on creating a compare function which is being designed to comapre all the rows of one dataset to one specific row of another dataset

In [48]:
def compare(data1, data2, row_comp):
    """
    This is a comparsion function  
    data1: First dataset for comparsion, 1 set row
    data2: Second dataset for comparsion
    row_comp: The row we want compared
    """
    for i in range(len(data2)):
        print('Real data: ' + str(data1.loc[row_comp, 'Age'])+ '. Fake data: ' +str(data2.loc[i, 'Age']))